In [41]:
import json
import os
import random
import pickle
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#from nltk.stem import SnowballStemmer
#stemmer = SnowballStemmer('spanish')

In [2]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = np.array(data['train_x'])
train_y=np.array(data['train_y'])

In [15]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.matrix(bag))

In [23]:
test=bow('Which mopeds do you have for sales?',words)
print(test.shape)
print(test)

(1, 48)
[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 0 0 1]]


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [11]:
model = Sequential()
model.add(Dense(25, input_dim=train_x.shape[1]))          # densidad de la primera capa de neurona y tipo de entrada
model.add(Dropout(0.5))                                        # convierte a 0 la mitad de 1 en el entrenamiento
model.add(Dense(25))                                           # densidad de la primera capa de neurona
model.add(Dropout(0.5))                                        # convierte a 0 la mitad de 1 en el entrenamiento
model.add(Dense(train_y.shape[1], activation='softmax'))  # densidad de la salida

In [6]:
model = load_model(os.path.join('./','model.h5'))

In [17]:
p=model.predict(test)

In [18]:
print(p)

[[  5.96715433e-09   2.53333066e-09   8.64720988e-08   9.99991298e-01
    1.19370313e-07   7.49474293e-06   5.40863084e-07   5.24998484e-07
    2.52456278e-08]]


In [20]:
idclass=np.argmax(p)
print(idclass)

3


In [21]:
print(classes[idclass])

mopeds


In [26]:
test=bow('When do you open?',words)
print(test.shape)
print(test)

(1, 48)
[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 1 0 0 0 1]]


In [27]:
p=model.predict(test)
print(p)
idclass=np.argmax(p)
print(classes[idclass])

[[  1.23793230e-04   6.77201433e-06   6.45547450e-01   1.61597761e-03
    3.52610588e-01   2.53145208e-05   3.36515295e-05   5.66187282e-06
    3.07604569e-05]]
hours


In [28]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [44]:
predictedClass=classes[idclass]
for intent in intents['intents']:
    if intent["tag"]==predictedClass:
        sizeResponses=len(intent["responses"])
        chooseIdResponse=random.randint(0,sizeResponses)
        print(intent["responses"][chooseIdResponse])


Our hours are 9am-9pm every day


In [32]:
def predictClass(sentence,model,classes,words):
    sentenceBow=bow(sentence,words)
    p=model.predict(sentenceBow)
    idClass=np.argmax(p)
    pClass=np.max(p)
    className=classes[idClass]
    return className,idClass,pClass

In [45]:
sentence="Do you accept visa?"
cn,idc,pc=predictClass(sentence,model,classes,words)
print(idc,cn,pc)

5 payments 0.992783


In [47]:
sentence="Can I pay with Mastercard?"
cn,idc,pc=predictClass(sentence,model,classes,words)
print(idc,cn,pc)

6 rental 0.473781


In [66]:
def chooseRandom(responses):
    sizeResponses=len(responses)
    chooseIdResponse=random.randint(0,sizeResponses-1)
    return responses[chooseIdResponse]

def chooseResponse(predictedClass):
    for intent in intents['intents']:
        if intent["tag"]==predictedClass:
            return chooseRandom(intent["responses"])

In [67]:
response=chooseResponse(cn)
print(response)

Are you looking to rent today or later this week?


In [69]:
def chat(sentence):
    cn,idc,pc=predictClass(sentence,model,classes,words)
    print(idc,cn,pc)
    if pc<0.6:
        return chooseRandom(["I don't understand your sentence.",
                             "What do you wanted to mean?",
                             "Could you please repeat?"])
    return chooseResponse(cn)

In [70]:
print(chat("Good morning"))

1 greeting 0.994255
Good to see you again


In [71]:
print(chat("do you rent mopets?"))

3 mopeds 0.923053
We rent Yamaha, Piaggio and Vespa mopeds


In [72]:
print(chat("When do you open?"))

2 hours 0.645547
Our hours are 9am-9pm every day


In [73]:
print(chat("are you open how?"))

2 hours 0.685815
We're open every day 9am-9pm


In [74]:
print(chat("I would like to rent a piaggio."))

6 rental 0.99998
Are you looking to rent today or later this week?


In [75]:
print(chat("I would like to rent this week."))

6 rental 0.999951
Are you looking to rent today or later this week?


In [81]:
print(chat("Can I pay with Mastercard?"))

6 rental 0.473781
What do you wanted to mean?


In [82]:
print(chat("to week I would rent like this ."))

6 rental 0.999951
Are you looking to rent today or later this week?
